In [89]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV


#### First we will start by importing the data

In the notebook we work with the ghost state approach. Therefore our dataframe will consist of all the data for the previous elections for all states, concatenated in a single dataframe.

We will begin by reading the .csv files for the different states and concatenate them into a single dataframe.

#### Construct the base dataframe

In [2]:
base_df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])

In [3]:
display(base_df.head())

,month_10,month_11,month_9,republican,Year,Rep_House_Prop,State,Result,rep_loyalty,popular_vote_percentage,density,RDI
0,51.797513,51.909088,51.657848,1,1988,0.200000,Georgia,1,0.5,53.37,106.259013,93086.74
1,37.538311,38.412820,37.298773,0,1988,0.800000,Georgia,0,0.5,45.65,106.259013,93086.74
2,41.056567,39.511030,39.876493,0,1992,0.900000,Georgia,1,0.7,43.01,113.717367,100758.19
3,35.651345,35.969860,38.271617,1,1992,0.100000,Georgia,0,0.7,37.45,113.717367,100758.19
4,46.538179,44.819709,45.546053,0,1996,0.636364,Georgia,0,0.5,49.23,123.352989,108448.50


#### Splitting the data into train and test data (test data 2020 observations).

In [4]:
train_df = base_df[base_df.Year != 2020]
test_df = base_df[base_df.Year == 2020]

In [5]:
display(train_df.head())
display(test_df.head())

,month_10,month_11,month_9,republican,Year,Rep_House_Prop,State,Result,rep_loyalty,popular_vote_percentage,density,RDI
0,51.797513,51.909088,51.657848,1,1988,0.200000,Georgia,1,0.5,53.37,106.259013,93086.74
1,37.538311,38.412820,37.298773,0,1988,0.800000,Georgia,0,0.5,45.65,106.259013,93086.74
2,41.056567,39.511030,39.876493,0,1992,0.900000,Georgia,1,0.7,43.01,113.717367,100758.19
3,35.651345,35.969860,38.271617,1,1992,0.100000,Georgia,0,0.7,37.45,113.717367,100758.19
4,46.538179,44.819709,45.546053,0,1996,0.636364,Georgia,0,0.5,49.23,123.352989,108448.50


,month_10,month_11,month_9,republican,Year,Rep_House_Prop,State,Result,rep_loyalty,popular_vote_percentage,density,RDI
16,46.657139,47.298710,47.170450,1,2020,0.714286,Georgia,1,1.0,46.525,180.617755,163524.50
17,47.604381,48.407587,45.906192,0,2020,0.285714,Georgia,0,1.0,52.460,180.617755,163524.50
16,46.311941,47.027987,46.593060,1,2020,0.785714,North Carolina,1,0.8,46.525,197.170250,161835.04
17,48.855849,48.848577,47.920197,0,2020,0.214286,North Carolina,0,0.8,52.460,197.170250,161835.04
12,66.209900,62.336910,66.226278,1,2020,1.000000,Wyoming,1,1.0,46.525,5.796735,10593.19


In [6]:
train_df = train_df.drop(['Year', 'State'], axis = 1)
test_df = test_df.drop(['Year', 'State'], axis = 1)

In [7]:
display(train_df.head())
display(test_df.head())

,month_10,month_11,month_9,republican,Rep_House_Prop,Result,rep_loyalty,popular_vote_percentage,density,RDI
0,51.797513,51.909088,51.657848,1,0.200000,1,0.5,53.37,106.259013,93086.74
1,37.538311,38.412820,37.298773,0,0.800000,0,0.5,45.65,106.259013,93086.74
2,41.056567,39.511030,39.876493,0,0.900000,1,0.7,43.01,113.717367,100758.19
3,35.651345,35.969860,38.271617,1,0.100000,0,0.7,37.45,113.717367,100758.19
4,46.538179,44.819709,45.546053,0,0.636364,0,0.5,49.23,123.352989,108448.50


,month_10,month_11,month_9,republican,Rep_House_Prop,Result,rep_loyalty,popular_vote_percentage,density,RDI
16,46.657139,47.298710,47.170450,1,0.714286,1,1.0,46.525,180.617755,163524.50
17,47.604381,48.407587,45.906192,0,0.285714,0,1.0,52.460,180.617755,163524.50
16,46.311941,47.027987,46.593060,1,0.785714,1,0.8,46.525,197.170250,161835.04
17,48.855849,48.848577,47.920197,0,0.214286,0,0.8,52.460,197.170250,161835.04
12,66.209900,62.336910,66.226278,1,1.000000,1,1.0,46.525,5.796735,10593.19


In [8]:
train, val = train_test_split(train_df, train_size = 0.8, random_state=56)
X_train = train.drop('Result', axis=1)#.values
y_train = train['Result']#.values
X_val = val.drop('Result', axis=1)#.values
y_val = val['Result']#.values
X_test = test_df.drop('Result', axis=1)#.values
y_test = test_df['Result']#.values

#### Modelling phase

In [9]:
def modelling(X_train, X_val, y_train, y_val, models, scaler=True, Poly=1, PCA_comp=0):
    # pca is the total variance explained required. 0 means that we do not want to perform pca
    if 'State' in X_train.columns:
        X_train = X_train.drop('State', axis=1)
        X_val = X_val.drop('State', axis=1)
    if 'Year' in X_train.columns:
        X_train = X_train.drop('Year', axis=1)
        X_val = X_val.drop('Year', axis=1)
    if PCA_comp > 0 and not scaler:
        raise ValueError('We must normalize before performing PCA')
    if scaler:
        # print('Normalizing the data ...')
        scaler = MinMaxScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
    if PCA_comp > 0:
        # print('Performing PCA ...')
        pca = PCA(n_components=X_train.shape[1])
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_val = pca.transform(X_val)
        total_variance = np.cumsum(pca.explained_variance_ratio_)
        for i, variance in enumerate(total_variance):
            if variance > PCA_comp:
                break 
        X_train, X_val = X_train[:, :i+1], X_val[:, :i+1]
    if Poly > 1:
        # print('Polynomial transformation ...')
        poly_features = PolynomialFeatures(degree=Poly, include_bias=False).fit(X_train)
        X_train = poly_features.transform(X_train)
        X_val = poly_features.transform(X_val)
    accuracies = []
    auc = []
    # print('Modelling and gathering the predictions ..')
    for i, model in enumerate(models):
        model.fit(X_train, y_train)
        predictions = model.predict(X_val)
        accuracies.append(accuracy_score(y_val, predictions))
        try:
            auc.append(roc_auc_score(y_val, predictions))
        except ValueError:
            auc.append(1)
    return models, accuracies, auc, predictions

#### 1. Logistic Regression
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [61]:
models_considered = [LogisticRegressionCV(max_iter=1000, penalty='l1', solver='saga')]
models_11, accuracies_11, aucs_11, predictions_11 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=1, PCA_comp=0)

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
print("{0}: {1}".format(models_11[0], accuracies_11[0]))

PolynomialFeatures = 2, PCA = 0

In [12]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_12, accuracies_12, aucs_12, predictions_12 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=2, PCA_comp=0)

In [13]:
print("{0}: {1}".format(models_12[0], accuracies_12[0]))

LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga'): 0.8860759493670886


In [14]:
poly_features = PolynomialFeatures(degree=2, include_bias=False).fit(X_train)
scaler = MinMaxScaler().fit(X_train)
X_train1 = scaler.transform(X_train)
X_val1 = scaler.transform(X_val)
X_test1 = scaler.transform(X_test)

X_train_poly = poly_features.transform(X_train1)
X_val_poly = poly_features.transform(X_val1)
X_test_poly = poly_features.transform(X_test1)
model = LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga').fit(X_train_poly, y_train)
print(accuracy_score(y_val, model.predict(X_val_poly)))
print(accuracy_score(y_test, model.predict(X_test_poly)))
print(model.score(X_test_poly, y_test))

0.8860759493670886
0.96
0.96


PolynomialFeatures = 2, PCA = 0.9

In [15]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_13, accuracies_13, aucs_13, predictions_13 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)

In [16]:
print("{0}: {1}".format(models_13[0], accuracies_13[0]))

LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga'): 0.8860759493670886


PolynomialFeatures = 1, PCA = 0.9

In [17]:
models_considered = [LogisticRegressionCV(max_iter=1000, penalty='l1', solver='saga')]
models_14, accuracies_14, aucs_14, predictions_14 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)

In [18]:
print("{0}: {1}".format(models_14[0], accuracies_14[0]))

LogisticRegressionCV(max_iter=1000, penalty='l1', solver='saga'): 0.7911392405063291


#### 2. Decision Tree Classifier
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [25]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_21, accuracies_21, aucs_21, predictions_21 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=1, PCA_comp=0)
    print("{0}: {1}".format(models_21[0], accuracies_21[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.810126582278481
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.810126582278481
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8544303797468354
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8734177215189873
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.879746835443038
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8987341772151899
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8734177215189873
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.879746835443038
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8860759493670886
----------------------------------------------------

PolynomialFeatures = 2, PCA = 0

In [26]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_22, accuracies_22, aucs_22, predictions_22 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=2, PCA_comp=0)
    print("{0}: {1}".format(models_22[0], accuracies_22[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.810126582278481
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.810126582278481
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8544303797468354
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8544303797468354
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8670886075949367
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8544303797468354
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8734177215189873
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8670886075949367
--------------------------------------------------

PolynomialFeatures = 2, PCA = 0.9

In [27]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_23, accuracies_23, aucs_23, predictions_23 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
    print("{0}: {1}".format(models_23[0], accuracies_23[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.7911392405063291
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8164556962025317
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8227848101265823
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8354430379746836
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8924050632911392
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8607594936708861
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8544303797468354
------------------------------------------------

PolynomialFeatures = 1, PCA = 0.9

In [28]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_24, accuracies_24, aucs_24, predictions_24 = modelling(X_train, X_val, y_train, y_val, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)
    print("{0}: {1}".format(models_24[0], accuracies_24[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.7848101265822784
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.7848101265822784
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8227848101265823
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8481012658227848
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8227848101265823
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8291139240506329
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8544303797468354
------------------------------------------------

#### 3. Random Forrest Classifier

In [41]:
# Number of trees in random forest
n_estimators_over = [50, 90, 110, 150, 200, 300, 400, 500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth_over = [int(x) for x in np.linspace(1, 10, num=6)]
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators_over,
               'max_features': max_features,
               'max_depth': max_depth_over,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,

               }
# Use the random grid to search for best hyperparameters

rf = RandomForestClassifier()
random_model = GridSearchCV(estimator=rf, param_grid=random_grid, verbose=1,
                                    scoring='accuracy',
                                    n_jobs=-1, return_train_score=True, cv=2, refit=True)
random_model.fit(X_train, y_train)
dict_rf = cross_validate(RandomForestClassifier(**random_model.best_params_), X_train, y_train, cv=9, scoring='accuracy')
rf_initial_accuracy = np.mean(dict_rf['test_score'])

Fitting 2 folds for each of 1728 candidates, totalling 3456 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 530 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 880 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1880 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2530 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3280 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed:  3.4min finished


In [42]:
random_model.best_params_

{'bootstrap': False,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 110}

In [43]:
rf_initial_accuracy

0.8904761904761904

#### 4. Ada Boost Classifier

In [53]:
depths = np.linspace(1,10,10)

# Base Estimator
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 15)]
# Learning Rate
learning_rate = [1e-3,1e-2,1e-1,1,10]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate}
scores = []
params_scaled = []
for depth in depths:
    # Create a based model
    ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = depth))
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = ada, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 2, scoring = 'accuracy')
    grid_search.fit(X_train, y_train)
    dict_boosting = cross_validate(AdaBoostClassifier(**grid_search.best_params_), X_train, y_train, cv=9, scoring='accuracy', n_jobs=-1)
    boosting_accuracy = np.mean(dict_boosting['test_score'])
    print('With components ', depth, 'the CV score is ', boosting_accuracy)
    print(grid_search.best_params_)

Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    5.1s finished


With components  1.0 the CV score is  0.8746031746031746
{'learning_rate': 0.1, 'n_estimators': 64}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    5.9s finished


With components  2.0 the CV score is  0.8285714285714286
{'learning_rate': 0.01, 'n_estimators': 159}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


...

With components  8.0 the CV score is  0.8714285714285714
{'learning_rate': 1, 'n_estimators': 77}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    1.3s finished


With components  9.0 the CV score is  0.8380952380952381
{'learning_rate': 0.01, 'n_estimators': 172}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.6s finished


With components  10.0 the CV score is  0.8253968253968255
{'learning_rate': 0.001, 'n_estimators': 91}


### Re-do everything with cross-validation only, no splitting in the traning set

In [54]:
df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])

In [55]:
train_df = df[base_df.Year != 2020]
test_df = df[base_df.Year == 2020]

In [56]:
train_df = train_df.drop(['Year', 'State'], axis = 1)
test_df = test_df.drop(['Year', 'State'], axis = 1)

In [57]:
X_train = train_df.drop('Result', axis=1)#.values
y_train = train_df['Result']#.values
X_test = test_df.drop('Result', axis=1)#.values
y_test = test_df['Result']#.values

In [71]:
def modelling(X_train, y_train, models, scaler=True, Poly=1, PCA_comp=0):
    # pca is the total variance explained required. 0 means that we do not want to perform pca
    if 'State' in X_train.columns:
        X_train = X_train.drop('State', axis=1)
    if 'Year' in X_train.columns:
        X_train = X_train.drop('Year', axis=1)
    if PCA_comp > 0 and not scaler:
        raise ValueError('We must normalize before performing PCA')
    if scaler:
        # print('Normalizing the data ...')
        scaler = MinMaxScaler().fit(X_train)
        X_train = scaler.transform(X_train)
    if PCA_comp > 0:
        # print('Performing PCA ...')
        pca = PCA(n_components=X_train.shape[1])
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        total_variance = np.cumsum(pca.explained_variance_ratio_)
        for i, variance in enumerate(total_variance):
            if variance > PCA_comp:
                break 
        X_train = X_train[:, :i+1]
    if Poly > 1:
        # print('Polynomial transformation ...')
        poly_features = PolynomialFeatures(degree=Poly, include_bias=False).fit(X_train)
        X_train = poly_features.transform(X_train)
    model_cv_accuracies = []
    # print('Modelling and gathering the predictions ..')
    for i, model in enumerate(models):
        model_cv = cross_validate(model, X_train, y_train, cv=5, scoring='accuracy')
        model_cv_accuracy = np.mean(model_cv['test_score'])
        model_cv_accuracies.append(model_cv_accuracy)
    return models, model_cv_accuracies

#### 1. Logistic Regression
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [73]:
models_considered = [LogisticRegressionCV(max_iter=1000, penalty='l1', solver='saga')]
models_11, cv_accuracies_11 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0)
print(cv_accuracies_11)

[0.8705877610255583]


PolynomialFeatures = 2, PCA = 0

In [74]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_12, cv_accuracies_12 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0)
print(cv_accuracies_12)

[0.879464645650246]


In [75]:
poly_features = PolynomialFeatures(degree=2, include_bias=False).fit(X_train)
scaler = MinMaxScaler().fit(X_train)
X_train1 = scaler.transform(X_train)
X_test1 = scaler.transform(X_test)

X_train_poly = poly_features.transform(X_train1)
X_test_poly = poly_features.transform(X_test1)
model = LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')
print(np.mean(cross_validate(model, X_train_poly, y_train, cv=5, scoring='accuracy')['test_score']))
model.fit(X_train_poly, y_train)
print(model.score(X_test_poly, y_test))

0.879464645650246
0.96


PolynomialFeatures = 2, PCA = 0.9

In [77]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_13, cv_accuracies_13 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
print(cv_accuracies_13)

[0.8476900749818593]


PolynomialFeatures = 1, PCA = 0.9

In [78]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_14, cv_accuracies_14 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)
print(cv_accuracies_14)

[0.7640248327017657]


#### 2. Decision Tree Classifier
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [79]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_21, cv_accuracies_21 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0)
    print("{0}: {1}".format(models_21[0], cv_accuracies_21[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.8262033379021204
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8262033379021204
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8604611787470773
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8693058131097317
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8451664919777473
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8489881480287028
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.850253970813513
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8527775538176247
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8363137950495847
-------------------------------------------------

PolynomialFeatures = 2, PCA = 0

In [80]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_22, cv_accuracies_22 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0)
    print("{0}: {1}".format(models_22[0], cv_accuracies_22[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.8300088688220593
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8300088688220593
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8414254615818753
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8248730145932436
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8286543578166572
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.843916794323954
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8489639603321777
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8400951382729985
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8349995968717245
-------------------------------------------------

PolynomialFeatures = 2, PCA = 0.9

In [82]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_23, cv_accuracies_23 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
    print("{0}: {1}".format(models_23[0], cv_accuracies_23[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.7804402160767556
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8032895267274046
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8312021285172942
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8020075788115779
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8209787954527131
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.815947754575506
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8096025155204385
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8184874627106347
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.815947754575506
--------------------------------------------------

PolynomialFeatures = 1, PCA = 0.9

In [83]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_24, cv_accuracies_24 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)
    print("{0}: {1}".format(models_24[0], cv_accuracies_24[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.765339030879626
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.7589292913004918
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.7881157784406998
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8121664113520921
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8147383697492543
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8147061194872208
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.828670482947674
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8197774731919697
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8184794001451262
--------------------------------------------------

#### 3. Random Forrest Classifier

In [84]:
# Number of trees in random forest
n_estimators_over = [50, 90, 110, 150, 200, 300, 400, 500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth_over = [int(x) for x in np.linspace(1, 10, num=6)]
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators_over,
               'max_features': max_features,
               'max_depth': max_depth_over,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,

               }
# Use the random grid to search for best hyperparameters

rf = RandomForestClassifier()
random_model = GridSearchCV(estimator=rf, param_grid=random_grid, verbose=1,
                                    scoring='accuracy',
                                    n_jobs=-1, return_train_score=True, cv=2, refit=True)
random_model.fit(X_train, y_train)
dict_rf = cross_validate(RandomForestClassifier(**random_model.best_params_), X_train, y_train, cv=9, scoring='accuracy')
rf_initial_accuracy = np.mean(dict_rf['test_score'])

Fitting 2 folds for each of 1728 candidates, totalling 3456 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed:  3.7min finished


In [85]:
print(random_model.best_params_)
print(rf_initial_accuracy)

{'bootstrap': True, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
0.8934169278996866


In [191]:
rf = RandomForestClassifier(bootstrap = True, max_depth = 8, max_features = 'sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.92

#### 4. Ada Boost Classifier

In [86]:
depths = np.linspace(1,10,10)

# Base Estimator
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 15)]
# Learning Rate
learning_rate = [1e-3,1e-2,1e-1,1,10]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate}
scores = []
params_scaled = []
for depth in depths:
    # Create a based model
    ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = depth))
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = ada, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 2, scoring = 'accuracy')
    grid_search.fit(X_train, y_train)
    dict_boosting = cross_validate(AdaBoostClassifier(**grid_search.best_params_), X_train, y_train, cv=9, scoring='accuracy', n_jobs=-1)
    boosting_accuracy = np.mean(dict_boosting['test_score'])
    print('With components ', depth, 'the CV score is ', boosting_accuracy)
    print(grid_search.best_params_)

Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    5.3s finished


With components  1.0 the CV score is  0.8871037965865551
{'learning_rate': 1, 'n_estimators': 132}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    6.5s finished


With components  2.0 the CV score is  0.8503715314060141
{'learning_rate': 0.1, 'n_estimators': 23}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


...

With components  8.0 the CV score is  0.8807616393823291
{'learning_rate': 1, 'n_estimators': 37}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    1.2s finished


With components  9.0 the CV score is  0.8845930570068501
{'learning_rate': 0.1, 'n_estimators': 105}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.6s finished


With components  10.0 the CV score is  0.8402269824683617
{'learning_rate': 0.01, 'n_estimators': 159}


In [189]:
ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 7), learning_rate = 1, n_estimators = 105).fit(X_train, y_train)
ada.score(X_test,y_test)

0.92

#### 5. Fully Connected Neural Network

In [176]:
df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])

In [177]:
train_df = df[base_df.Year != 2020]
test_df = df[base_df.Year == 2020]

In [178]:
train_df = train_df.drop(['Year', 'State'], axis = 1)
test_df = test_df.drop(['Year', 'State'], axis = 1)

In [179]:
X_train = train_df.drop('Result', axis=1)#.values
y_train = train_df['Result']#.values
X_test = test_df.drop('Result', axis=1)#.values
y_test = test_df['Result']#.values

In [180]:
display(X_train.head())
display(X_test.head())

,month_10,month_11,month_9,republican,Rep_House_Prop,rep_loyalty,popular_vote_percentage,density,RDI
0,51.797513,51.909088,51.657848,1,0.200000,0.5,53.37,106.259013,93086.74
1,37.538311,38.412820,37.298773,0,0.800000,0.5,45.65,106.259013,93086.74
2,41.056567,39.511030,39.876493,0,0.900000,0.7,43.01,113.717367,100758.19
3,35.651345,35.969860,38.271617,1,0.100000,0.7,37.45,113.717367,100758.19
4,46.538179,44.819709,45.546053,0,0.636364,0.5,49.23,123.352989,108448.50


,month_10,month_11,month_9,republican,Rep_House_Prop,rep_loyalty,popular_vote_percentage,density,RDI
16,46.657139,47.298710,47.170450,1,0.714286,1.0,46.525,180.617755,163524.50
17,47.604381,48.407587,45.906192,0,0.285714,1.0,52.460,180.617755,163524.50
16,46.311941,47.027987,46.593060,1,0.785714,0.8,46.525,197.170250,161835.04
17,48.855849,48.848577,47.920197,0,0.214286,0.8,52.460,197.170250,161835.04
12,66.209900,62.336910,66.226278,1,1.000000,1.0,46.525,5.796735,10593.19


In [150]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [161]:
import tensorflow as tf

# Defining an Early Stopping Condition
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 50)
regularizer = 0.00015

In [174]:
model = tf.keras.models.Sequential(name = 'ANN')

model.add(tf.keras.layers.Dense(6, activation='relu', input_shape=(9,), name='hidden1', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(6, activation='relu', name='hidden2', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4, activation='relu', name='hidden3', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output'))

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=2, validation_split=0.2, callbacks=[callback])

Train on 630 samples, validate on 158 samples
Epoch 1/50
630/630 - 1s - loss: 0.9394 - accuracy: 0.4952 - val_loss: 0.8899 - val_accuracy: 0.4747
Epoch 2/50
630/630 - 0s - loss: 0.8626 - accuracy: 0.4825 - val_loss: 0.7851 - val_accuracy: 0.4557
Epoch 3/50
630/630 - 0s - loss: 0.7792 - accuracy: 0.4667 - val_loss: 0.7226 - val_accuracy: 0.4747
Epoch 4/50
630/630 - 0s - loss: 0.7192 - accuracy: 0.4889 - val_loss: 0.6826 - val_accuracy: 0.5316
Epoch 5/50
630/630 - 0s - loss: 0.7033 - accuracy: 0.5111 - val_loss: 0.6488 - val_accuracy: 0.6329
Epoch 6/50
630/630 - 0s - loss: 0.6645 - accuracy: 0.5698 - val_loss: 0.6058 - val_accuracy: 0.7152
Epoch 7/50
630/630 - 0s - loss: 0.6276 - accuracy: 0.6190 - val_loss: 0.5557 - val_accuracy: 0.7405
Epoch 8/50
630/630 - 0s - loss: 0.6025 - accuracy: 0.6429 - val_loss: 0.5150 - val_accuracy: 0.7785
Epoch 9/50
630/630 - 0s - loss: 0.5643 - accuracy: 0.6746 - val_loss: 0.4782 - val_accuracy: 0.7848
Epoch 10/50
630/630 - 0s - loss: 0.5476 - accuracy: 0.

In [175]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy={test_accuracy}')

100/1 - 0s - loss: 0.1440 - accuracy: 0.9300
Test accuracy=0.9300000071525574


In [172]:
model_simple = tf.keras.models.Sequential(name = 'ANN')

model_simple.add(tf.keras.layers.Dense(6, activation='relu', input_shape=(9,), name='hidden1'))
model_simple.add(tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output'))

model_simple.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
history = model_simple.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_split=0.3)

Train on 551 samples, validate on 237 samples
Epoch 1/100
551/551 [==============================] - 1s 2ms/sample - loss: 0.6569 - accuracy: 0.6388 - val_loss: 0.5806 - val_accuracy: 0.7173
Epoch 2/100
551/551 [==============================] - 0s 180us/sample - loss: 0.6185 - accuracy: 0.6733 - val_loss: 0.5525 - val_accuracy: 0.7553
Epoch 3/100
551/551 [==============================] - 0s 149us/sample - loss: 0.5883 - accuracy: 0.7060 - val_loss: 0.5285 - val_accuracy: 0.7848
Epoch 4/100
551/551 [==============================] - 0s 155us/sample - loss: 0.5619 - accuracy: 0.7332 - val_loss: 0.5084 - val_accuracy: 0.7932
Epoch 5/100
551/551 [==============================] - 0s 135us/sample - loss: 0.5401 - accuracy: 0.7586 - val_loss: 0.4910 - val_accuracy: 0.7975
Epoch 6/100
551/551 [==============================] - 0s 149us/sample - loss: 0.5210 - accuracy: 0.7822 - val_loss: 0.4752 - val_accuracy: 0.8059
Epoch 7/100
551/551 [==============================] - 0s 140us/sample - l

551/551 [==============================] - 0s 137us/sample - loss: 0.2677 - accuracy: 0.8838 - val_loss: 0.2575 - val_accuracy: 0.8692
Epoch 57/100
551/551 [==============================] - 0s 134us/sample - loss: 0.2667 - accuracy: 0.8857 - val_loss: 0.2569 - val_accuracy: 0.8692
Epoch 58/100
551/551 [==============================] - 0s 135us/sample - loss: 0.2660 - accuracy: 0.8857 - val_loss: 0.2571 - val_accuracy: 0.8692
Epoch 59/100
551/551 [==============================] - 0s 134us/sample - loss: 0.2654 - accuracy: 0.8838 - val_loss: 0.2570 - val_accuracy: 0.8692
Epoch 60/100
551/551 [==============================] - 0s 131us/sample - loss: 0.2645 - accuracy: 0.8820 - val_loss: 0.2559 - val_accuracy: 0.8692
Epoch 61/100
551/551 [==============================] - 0s 130us/sample - loss: 0.2637 - accuracy: 0.8875 - val_loss: 0.2553 - val_accuracy: 0.8692
Epoch 62/100
551/551 [==============================] - 0s 133us/sample - loss: 0.2631 - accuracy: 0.8857 - val_loss: 0.2548 

In [173]:
test_loss1, test_accuracy1 = model_simple.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy={test_accuracy1}')

100/1 - 0s - loss: 0.0920 - accuracy: 0.9400
Test accuracy=0.9399999976158142


We have done extensive model selection in the sections above. Nevetheless, we have identified a flaw with the above modeling approach. The flaw arises from the construction of the cross-validation set. Since for every year the data set contains two observations (one for the democrat candidate, one for the republican) if once of the two observations is in the training and the oter in the CV sets respectively than the model will easily recognize and memorize the pattern and therefore the model becomes near deterministic, i.e. there is a large information leakage. 

In order to prevent this issue, the first approach we will try is to get rid of half the observations, in other words we will only consider the observations for one (the republican candidate) for each election year.

### Re-do everything with the reduced dataframe in order to prevent information leakage.

In [200]:
df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])
df_republican = df[df.republican != 0]

In [201]:
train_df = df_republican[df_republican.Year != 2020]
test_df = df_republican[df_republican.Year == 2020]

In [202]:
train_df = train_df.drop(['Year', 'State'], axis = 1)
test_df = test_df.drop(['Year', 'State'], axis = 1)

In [203]:
X_train = train_df.drop('Result', axis=1)#.values
y_train = train_df['Result']#.values
X_test = test_df.drop('Result', axis=1)#.values
y_test = test_df['Result']#.values

In [206]:
display(X_train.shape)
display(X_test.shape)

(394, 9)

(50, 9)

In [207]:
def modelling(X_train, y_train, models, scaler=True, Poly=1, PCA_comp=0):
    # pca is the total variance explained required. 0 means that we do not want to perform pca
    if 'State' in X_train.columns:
        X_train = X_train.drop('State', axis=1)
    if 'Year' in X_train.columns:
        X_train = X_train.drop('Year', axis=1)
    if PCA_comp > 0 and not scaler:
        raise ValueError('We must normalize before performing PCA')
    if scaler:
        # print('Normalizing the data ...')
        scaler = MinMaxScaler().fit(X_train)
        X_train = scaler.transform(X_train)
    if PCA_comp > 0:
        # print('Performing PCA ...')
        pca = PCA(n_components=X_train.shape[1])
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        total_variance = np.cumsum(pca.explained_variance_ratio_)
        for i, variance in enumerate(total_variance):
            if variance > PCA_comp:
                break 
        X_train = X_train[:, :i+1]
    if Poly > 1:
        # print('Polynomial transformation ...')
        poly_features = PolynomialFeatures(degree=Poly, include_bias=False).fit(X_train)
        X_train = poly_features.transform(X_train)
    model_cv_accuracies = []
    # print('Modelling and gathering the predictions ..')
    for i, model in enumerate(models):
        model_cv = cross_validate(model, X_train, y_train, cv=5, scoring='accuracy')
        model_cv_accuracy = np.mean(model_cv['test_score'])
        model_cv_accuracies.append(model_cv_accuracy)
    return models, model_cv_accuracies

#### 1. Logistic Regression
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [209]:
models_considered = [LogisticRegressionCV(max_iter=2000, penalty='l1', solver='saga')]
models_11, cv_accuracies_11 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0)
print(cv_accuracies_11)

[0.8858487504057123]


PolynomialFeatures = 2, PCA = 0

In [210]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_12, cv_accuracies_12 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0)
print(cv_accuracies_12)

[0.8808179162609543]


In [211]:
poly_features = PolynomialFeatures(degree=2, include_bias=False).fit(X_train)
scaler = MinMaxScaler().fit(X_train)
X_train1 = scaler.transform(X_train)
X_test1 = scaler.transform(X_test)

X_train_poly = poly_features.transform(X_train1)
X_test_poly = poly_features.transform(X_test1)
model = LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')
print(np.mean(cross_validate(model, X_train_poly, y_train, cv=5, scoring='accuracy')['test_score']))
model.fit(X_train_poly, y_train)
print(model.score(X_test_poly, y_test))

0.8808179162609543
0.98


PolynomialFeatures = 2, PCA = 0.9

In [213]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_13, cv_accuracies_13 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
print(cv_accuracies_13)

[0.8808179162609543]


PolynomialFeatures = 1, PCA = 0.9

In [214]:
models_considered = [LogisticRegressionCV(max_iter=5000, penalty='l1', solver='saga')]
models_14, cv_accuracies_14 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)
print(cv_accuracies_14)

[0.8884453099642974]


#### 2. Decision Tree Classifier
- PolynomialFeatures = 1, PCA = 0
- PolynomialFeatures = 2, PCA = 0
- PolynomialFeatures = 2, PCA = 0.9
- PolynomialFeatures = 1, PCA = 0.9

PolynomialFeatures = 1, PCA = 0

In [215]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_21, cv_accuracies_21 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0)
    print("{0}: {1}".format(models_21[0], cv_accuracies_21[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.840181759169101
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8173644920480363
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8426160337552743
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8630314832846478
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8477117818889971
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8554365465757872
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8071080817916261
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8096397273612463
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8348912690684843
-------------------------------------------------

PolynomialFeatures = 2, PCA = 0

In [216]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_22, cv_accuracies_22 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0)
    print("{0}: {1}".format(models_22[0], cv_accuracies_22[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.8807530022719897
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8503083414475819
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8527426160337553
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.858033106134372
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8350535540408958
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8401493021746186
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8478091528724441
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8299253489126908
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.850308341447582
--------------------------------------------------

PolynomialFeatures = 2, PCA = 0.9

In [217]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_23, cv_accuracies_23 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=2, PCA_comp=0.9)
    print("{0}: {1}".format(models_23[0], cv_accuracies_23[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.8757221681272315
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8731905225576112
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.8681272314183707
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8603700097370982
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.8604998377150276
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8503083414475819
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8299578059071729
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8273937033430704
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.8375202856215515
------------------------------------------------

PolynomialFeatures = 1, PCA = 0.9

In [218]:
depths = np.arange(1, 10, 1)
for depth in depths:
    models_considered = [DecisionTreeClassifier(max_depth=depth)]
    models_24, cv_accuracies_24 = modelling(X_train, y_train, models=models_considered, scaler=True, Poly=1, PCA_comp=0.9)
    print("{0}: {1}".format(models_24[0], cv_accuracies_24[0]))
    print('-'*55)

DecisionTreeClassifier(max_depth=1): 0.8757221681272315
-------------------------------------------------------
DecisionTreeClassifier(max_depth=2): 0.8782538136968517
-------------------------------------------------------
DecisionTreeClassifier(max_depth=3): 0.86053229470951
-------------------------------------------------------
DecisionTreeClassifier(max_depth=4): 0.8451152223304123
-------------------------------------------------------
DecisionTreeClassifier(max_depth=5): 0.840084388185654
-------------------------------------------------------
DecisionTreeClassifier(max_depth=6): 0.8578059071729957
-------------------------------------------------------
DecisionTreeClassifier(max_depth=7): 0.8527750730282376
-------------------------------------------------------
DecisionTreeClassifier(max_depth=8): 0.8374878286270692
-------------------------------------------------------
DecisionTreeClassifier(max_depth=9): 0.842583576760792
----------------------------------------------------

#### 3. Random Forrest Classifier

In [219]:
# Number of trees in random forest
n_estimators_over = [50, 90, 110, 150, 200, 300, 400, 500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth_over = [int(x) for x in np.linspace(1, 10, num=6)]
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators_over,
               'max_features': max_features,
               'max_depth': max_depth_over,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,

               }
# Use the random grid to search for best hyperparameters

rf = RandomForestClassifier()
random_model = GridSearchCV(estimator=rf, param_grid=random_grid, verbose=1,
                                    scoring='accuracy',
                                    n_jobs=-1, return_train_score=True, cv=2, refit=True)
random_model.fit(X_train, y_train)
dict_rf = cross_validate(RandomForestClassifier(**random_model.best_params_), X_train, y_train, cv=9, scoring='accuracy')
rf_initial_accuracy = np.mean(dict_rf['test_score'])

Fitting 2 folds for each of 1728 candidates, totalling 3456 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed:  3.5min finished


In [220]:
print(random_model.best_params_)
print(rf_initial_accuracy)

{'bootstrap': True, 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 50}
0.8910030537937514


In [226]:
rf = RandomForestClassifier(bootstrap = True, max_depth = 4, max_features = 'auto', min_samples_leaf = 1, min_samples_split = 3, n_estimators = 50)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.96

#### 4. Ada Boost Classifier

In [222]:
depths = np.linspace(1,10,10)

# Base Estimator
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 15)]
# Learning Rate
learning_rate = [1e-3,1e-2,1e-1,1,10]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate}
scores = []
params_scaled = []
for depth in depths:
    # Create a based model
    ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = depth))
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = ada, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 2, scoring = 'accuracy')
    grid_search.fit(X_train, y_train)
    dict_boosting = cross_validate(AdaBoostClassifier(**grid_search.best_params_), X_train, y_train, cv=9, scoring='accuracy', n_jobs=-1)
    boosting_accuracy = np.mean(dict_boosting['test_score'])
    print('With components ', depth, 'the CV score is ', boosting_accuracy)
    print(grid_search.best_params_)

Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    5.0s finished


With components  1.0 the CV score is  0.8784355179704018
{'learning_rate': 0.1, 'n_estimators': 77}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    5.7s finished


With components  2.0 the CV score is  0.8554733380314775
{'learning_rate': 0.01, 'n_estimators': 91}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


With components  9.0 the CV score is  0.8784355179704016
{'learning_rate': 0.1, 'n_estimators': 37}
Fitting 2 folds for each of 75 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s


With components  10.0 the CV score is  0.8531242659149636
{'learning_rate': 1, 'n_estimators': 10}


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.7s finished


In [225]:
ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 1), learning_rate = 0.1, n_estimators = 77).fit(X_train, y_train)
ada.score(X_test,y_test)

0.94

#### 5. Fully Connected Neural Network

In [228]:
df = pd.concat([pd.read_csv('states/'+file) for file in os.listdir('states/')])
df_republican = df[df.republican != 0]

In [229]:
train_df = df_republican[df_republican.Year != 2020]
test_df = df_republican[df_republican.Year == 2020]

In [230]:
train_df = train_df.drop(['Year', 'State'], axis = 1)
test_df = test_df.drop(['Year', 'State'], axis = 1)

In [231]:
X_train = train_df.drop('Result', axis=1)#.values
y_train = train_df['Result']#.values
X_test = test_df.drop('Result', axis=1)#.values
y_test = test_df['Result']#.values

In [232]:
display(X_train.head())
display(X_test.head())

,month_10,month_11,month_9,republican,Rep_House_Prop,rep_loyalty,popular_vote_percentage,density,RDI
0,51.797513,51.909088,51.657848,1,0.200000,0.5,53.37,106.259013,93086.74
3,35.651345,35.969860,38.271617,1,0.100000,0.7,37.45,113.717367,100758.19
5,40.574781,41.630290,40.888825,1,0.363636,0.5,40.72,123.352989,108448.50
7,48.459137,50.952625,44.657240,1,0.727273,0.7,47.87,138.411248,115406.50
8,54.915037,54.194542,55.351526,1,0.727273,0.8,50.73,147.528675,123452.01


,month_10,month_11,month_9,republican,Rep_House_Prop,rep_loyalty,popular_vote_percentage,density,RDI
16,46.657139,47.298710,47.170450,1,0.714286,1.0,46.525,180.617755,163524.50
16,46.311941,47.027987,46.593060,1,0.785714,0.8,46.525,197.170250,161835.04
12,66.209900,62.336910,66.226278,1,1.000000,1.0,46.525,5.796735,10593.19
16,42.653251,42.836953,43.010549,1,0.000000,0.0,46.525,146.642070,29220.57
16,43.585778,43.795763,43.493822,1,0.625000,0.4,46.525,89.335603,119627.08


In [233]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [234]:
import tensorflow as tf

# Defining an Early Stopping Condition
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 50)
regularizer = 0.00015

In [236]:
model = tf.keras.models.Sequential(name = 'ANN')

model.add(tf.keras.layers.Dense(6, activation='relu', input_shape=(9,), name='hidden1', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(6, activation='relu', name='hidden2', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4, activation='relu', name='hidden3', kernel_regularizer = tf.keras.regularizers.l1(regularizer)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output'))

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=40, batch_size=16, verbose=2, validation_split=0.2, callbacks=[callback])

Train on 315 samples, validate on 79 samples
Epoch 1/40
315/315 - 1s - loss: 0.7072 - accuracy: 0.5778 - val_loss: 0.5951 - val_accuracy: 0.8101
Epoch 2/40
315/315 - 0s - loss: 0.6874 - accuracy: 0.5810 - val_loss: 0.5897 - val_accuracy: 0.8101
Epoch 3/40
315/315 - 0s - loss: 0.7118 - accuracy: 0.6063 - val_loss: 0.5820 - val_accuracy: 0.8101
Epoch 4/40
315/315 - 0s - loss: 0.6703 - accuracy: 0.6317 - val_loss: 0.5707 - val_accuracy: 0.8101
Epoch 5/40
315/315 - 0s - loss: 0.6750 - accuracy: 0.6222 - val_loss: 0.5565 - val_accuracy: 0.8101
Epoch 6/40
315/315 - 0s - loss: 0.6860 - accuracy: 0.6508 - val_loss: 0.5485 - val_accuracy: 0.8101
Epoch 7/40
315/315 - 0s - loss: 0.6767 - accuracy: 0.6222 - val_loss: 0.5374 - val_accuracy: 0.8101
Epoch 8/40
315/315 - 0s - loss: 0.6365 - accuracy: 0.6667 - val_loss: 0.5244 - val_accuracy: 0.8228
Epoch 9/40
315/315 - 0s - loss: 0.6683 - accuracy: 0.6444 - val_loss: 0.5141 - val_accuracy: 0.8228
Epoch 10/40
315/315 - 0s - loss: 0.6442 - accuracy: 0.6

In [237]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy={test_accuracy}')

50/1 - 0s - loss: 0.1938 - accuracy: 0.9200
Test accuracy=0.9200000166893005


In [240]:
model_simple = tf.keras.models.Sequential(name = 'ANN')

model_simple.add(tf.keras.layers.Dense(6, activation='relu', input_shape=(9,), name='hidden1'))
model_simple.add(tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output'))

model_simple.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
history = model_simple.fit(X_train, y_train, epochs=150, batch_size=32, verbose=1, validation_split=0.3)

Train on 275 samples, validate on 119 samples
Epoch 1/150
275/275 [==============================] - 1s 3ms/sample - loss: 0.6253 - accuracy: 0.6036 - val_loss: 0.5266 - val_accuracy: 0.6891
Epoch 2/150
275/275 [==============================] - 0s 240us/sample - loss: 0.6074 - accuracy: 0.6291 - val_loss: 0.5124 - val_accuracy: 0.7227
Epoch 3/150
275/275 [==============================] - 0s 273us/sample - loss: 0.5915 - accuracy: 0.6945 - val_loss: 0.4990 - val_accuracy: 0.7647
Epoch 4/150
275/275 [==============================] - 0s 307us/sample - loss: 0.5763 - accuracy: 0.7309 - val_loss: 0.4871 - val_accuracy: 0.7983
Epoch 5/150
275/275 [==============================] - 0s 306us/sample - loss: 0.5625 - accuracy: 0.7636 - val_loss: 0.4757 - val_accuracy: 0.8151
Epoch 6/150
275/275 [==============================] - 0s 258us/sample - loss: 0.5487 - accuracy: 0.7782 - val_loss: 0.4651 - val_accuracy: 0.8235
Epoch 7/150
275/275 [==============================] - 0s 252us/sample - l

275/275 [==============================] - 0s 257us/sample - loss: 0.2906 - accuracy: 0.8764 - val_loss: 0.2628 - val_accuracy: 0.8739
Epoch 57/150
275/275 [==============================] - 0s 247us/sample - loss: 0.2892 - accuracy: 0.8764 - val_loss: 0.2621 - val_accuracy: 0.8739
Epoch 58/150
275/275 [==============================] - 0s 263us/sample - loss: 0.2876 - accuracy: 0.8764 - val_loss: 0.2612 - val_accuracy: 0.8739
Epoch 59/150
275/275 [==============================] - 0s 249us/sample - loss: 0.2862 - accuracy: 0.8764 - val_loss: 0.2604 - val_accuracy: 0.8739
Epoch 60/150
275/275 [==============================] - 0s 227us/sample - loss: 0.2848 - accuracy: 0.8764 - val_loss: 0.2598 - val_accuracy: 0.8739
Epoch 61/150
275/275 [==============================] - 0s 234us/sample - loss: 0.2835 - accuracy: 0.8764 - val_loss: 0.2592 - val_accuracy: 0.8739
Epoch 62/150
275/275 [==============================] - 0s 262us/sample - loss: 0.2823 - accuracy: 0.8764 - val_loss: 0.2587 

275/275 [==============================] - 0s 238us/sample - loss: 0.2493 - accuracy: 0.8873 - val_loss: 0.2555 - val_accuracy: 0.8824
Epoch 112/150
275/275 [==============================] - 0s 247us/sample - loss: 0.2489 - accuracy: 0.8909 - val_loss: 0.2557 - val_accuracy: 0.8824
Epoch 113/150
275/275 [==============================] - 0s 239us/sample - loss: 0.2486 - accuracy: 0.8909 - val_loss: 0.2557 - val_accuracy: 0.8824
Epoch 114/150
275/275 [==============================] - 0s 220us/sample - loss: 0.2484 - accuracy: 0.8873 - val_loss: 0.2559 - val_accuracy: 0.8824
Epoch 115/150
275/275 [==============================] - 0s 191us/sample - loss: 0.2479 - accuracy: 0.8909 - val_loss: 0.2561 - val_accuracy: 0.8824
Epoch 116/150
275/275 [==============================] - 0s 188us/sample - loss: 0.2476 - accuracy: 0.8909 - val_loss: 0.2563 - val_accuracy: 0.8824
Epoch 117/150
275/275 [==============================] - 0s 189us/sample - loss: 0.2472 - accuracy: 0.8945 - val_loss: 0

In [241]:
test_loss1, test_accuracy1 = model_simple.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy={test_accuracy1}')

50/1 - 0s - loss: 0.1635 - accuracy: 0.9800
Test accuracy=0.9800000190734863
